In [5]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn import preprocessing
pd.set_option("display.max_columns", 100)
from sklearn.metrics import f1_score
DATA_PATH = Path.cwd().parent / "ML project" 
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.impute import MissingIndicator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import Normalizer
RANDOM_SEED = 6    # Set a random seed for reproducibility!

In [6]:

from math import cos, asin, sqrt, pi

def distanceLong(lat1, lon1, lat2, lon2):
    p = pi/180
    latDiff = (lat2-lat1)*p
    lonDiff = (lon2-lon1)*p
    coslat1 = np.cos(lat1*p)
    coslat2 = np.cos(lat2*p)
    coslondiff = np.cos(lonDiff)
    coslatdiff = np.cos(latDiff)
    a = 0.5 - coslatdiff/2 + coslat1 * coslat2 * (1-coslondiff)/2
    return 12742 * (np.arcsin(a**0.5)) 

In [87]:

def formatdata(dataFrame):
     
    dataFrame['pickup_time'] = dataFrame['pickup_time']+ '.099'
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x:  x.replace("/","-"))
    dataFrame['drop_time'] = dataFrame['drop_time']+ '.099'

    pd.to_datetime(dataFrame['pickup_time'], format='%m-%d-%Y %H:%M:%S.%f', errors='ignore')
    pd.to_datetime(dataFrame['drop_time'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore')
    
    dataFrame['rush'] = (pd.to_datetime(dataFrame['pickup_time'], errors='ignore').dt.hour)
    dataFrame['rush'] = dataFrame['rush'].apply(lambda x: ((x >= 7 and x < 9) or (x >= 17 and x < 19)))
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m-%d-%Y %H:%M.%f')))
    dataFrame['drop_time'] = dataFrame['drop_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m/%d/%Y %H:%M.%f')))
    dataFrame['timediff'] =  dataFrame['drop_time'] - dataFrame['pickup_time']
   # dataFrame['timediff1'] =  dataFrame['timediff']**2
    dataFrame['distance'] = distanceLong(dataFrame['drop_lat'],dataFrame['drop_lon'],dataFrame['pick_lat'],dataFrame['pick_lon'])*100
    #dataFrame['distance'] = ((dataFrame['drop_lat']-dataFrame['pick_lat'])**2+(dataFrame['drop_lon']-dataFrame['pick_lon'])**2)**0.5
    #dataFrame['unit_meter_waiting_fair'] = dataFrame['meter_waiting_fare'] / dataFrame['meter_waiting']
    #print(dataFrame['unit_meter_waiting_fair'])
    
    #dataFrame['_fare'] = dataFrame['fare'] - dataFrame['additional_fare'] 
    #dataFrame['tripDuration'] = dataFrame['duration'] - dataFrame['meter_waiting']
    
    
    return dataFrame

 
bestDegree = 0
bestBias = False
bestInteraction = False
bestscore = 0

In [88]:

solver = 'lbfgs'
alpha = 0.2
hidden_layer_sizes=(10,2 )
random_state=2
max_iter= 10000000
activation='tanh'
#estimatorsX=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
 #                        max_iter=max_iter, activation=activation)

min_child_weight=1.5
max_depth=5
gamma=2.5
colsample_bytree=0.8
subsample=0.8
n_estimators=200
threads = 4
estimatorsXX = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )
params = { 'min_child_weight': [1.5,2,2.5],'max_depth':[8,9,7]}

#estimatorsY = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=6,learning_rate=0.2,
 #                          n_estimators=1000,random_state=42)

estimators = BaggingClassifier(base_estimator=estimatorsXX,n_estimators=10,max_samples=0.65,
                             max_features=1.0, random_state=1,bootstrap=False)
#estimators = VotingClassifier( [('xgb', estimatorsX), ('mlp', estimatorsY)], weights=[1,1])
#estimators = GridSearchCV(estimator=estimatorsXX, param_grid=params, cv=10)


train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)



train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]
features_df = formatdata(features_df)
features_df = features_df.fillna(features_df.mean())
numeric_cols = features_df.columns[features_df.dtypes != object].values



def removeColumns(npArry, columnNames):
    li = list(npArry)
    for i in columnNames:
        li.remove(i)
    return np.array(li)


def powerAll(data_frame, exceptList,power):
    allColumns = data_frame.columns.values
    for i in allColumns:
        if(i not in exceptList ):
            data_frame[i] = data_frame[i]**power
 

#nothing
exceptList = []
power = 1

numeric_cols = removeColumns(numeric_cols,['drop_time', 'pickup_time','drop_lat','drop_lon','pick_lat','pick_lon'
                                             ])  

#powerAll(features_df, exceptList,1)
#exceptList = features_df.columns[features_df.dtypes == "object"].values
#polyList1 = ['meter_waiting_fare','fare']
polyList1 = ['additional_fare','distance','meter_waiting']
polyList2 = ['timediff','distance']
polyList3 = ['meter_waiting','duration']
polyList4 = ['timediff','fare']
polyList5 = ['distance','fare']
polyList6 = ['meter_waiting_fare','additional_fare']
numeric_preprocessing_steps = Pipeline([ 
    ('standard_scaler', StandardScaler()),
])




feature_transformer2 = PolynomialFeatures(degree=4, include_bias=False,interaction_only=False)        
feature_transformer3 = PolynomialFeatures(degree=4, include_bias=True,interaction_only=False)
feature_transformer4 = PolynomialFeatures(degree=4, include_bias=True,interaction_only=False)
feature_transformer5 = PolynomialFeatures(degree=4, include_bias=True,interaction_only=False)
feature_transformer6 = PolynomialFeatures(degree=7, include_bias=False,interaction_only=False)
            
normalizer_transformer = Normalizer(norm ='l2')
            # create the preprocessor stage of final pipeline
            # each entry in the transformer list is a tuple of
            # (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ('poly2', feature_transformer2,polyList2),
        ('poly3', feature_transformer3,polyList3),
        ('poly4', feature_transformer4,polyList4),
        ('poly5', feature_transformer5,polyList5),
        ('poly6', feature_transformer6,polyList6),
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ("nomalizer", normalizer_transformer, numeric_cols)
    ],
    remainder = "drop"
)



C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [89]:
print(features_df.loc[213182176,'rush'])

True


In [64]:
best_df_list = []
statesList = range(100)
for i in statesList:    
    kf = KFold(n_splits=10, random_state=i, shuffle=True)


    kf.get_n_splits(features_df)
    test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                                   index_col="tripid")

    test_features_df = formatdata(test_features_df)
    powerAll(test_features_df, exceptList,power)

    bestscore = 0
    bestModelx = ""




    for train_index, test_index in kf.split(features_df):

        X_train, X_eval, y_train, y_eval = features_df.iloc[train_index], features_df.iloc[test_index],labels_df.iloc[train_index], labels_df.iloc[test_index]
        full_pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("estimators", estimators),
        ])
        full_pipeline.fit(X_train, y_train)
        preds = full_pipeline.predict(X_eval)
        y_preds = pd.DataFrame(
            preds,
            index = y_eval.index
        )
        test_probas = full_pipeline.predict(test_features_df)
        test_preds = pd.DataFrame(
            {'prediction':test_probas},
            index = test_features_df.index
        )
        f1 = f1_score(y_eval, y_preds)
        if(f1 > bestscore):
            bestscore = f1
            bestModelx = full_pipeline


    
    test_probas_state_best = bestModelx.predict(test_features_df)
    test_preds_state_best = pd.DataFrame(
        {'prediction':test_probas_state_best},
        index = test_features_df.index
    )
    best_df_list.append(test_preds_state_best)
    
    print(test_preds_state_best.prediction.value_counts())
    print('bestscore', bestscore)
    print("state", i)
    print('')

print("done")

KeyboardInterrupt: 

In [21]:
for i in best_df_list:
    print(i.prediction.value_counts())



1    8157
0     419
Name: prediction, dtype: int64
1    8134
0     442
Name: prediction, dtype: int64
1    8143
0     433
Name: prediction, dtype: int64
1    8143
0     433
Name: prediction, dtype: int64
1    8148
0     428
Name: prediction, dtype: int64
1    8142
0     434
Name: prediction, dtype: int64
1    8140
0     436
Name: prediction, dtype: int64
1    8147
0     429
Name: prediction, dtype: int64
1    8141
0     435
Name: prediction, dtype: int64
1    8149
0     427
Name: prediction, dtype: int64
1    8151
0     425
Name: prediction, dtype: int64
1    8154
0     422
Name: prediction, dtype: int64
1    8152
0     424
Name: prediction, dtype: int64
1    8143
0     433
Name: prediction, dtype: int64
1    8148
0     428
Name: prediction, dtype: int64
1    8161
0     415
Name: prediction, dtype: int64
1    8153
0     423
Name: prediction, dtype: int64
1    8150
0     426
Name: prediction, dtype: int64
1    8157
0     419
Name: prediction, dtype: int64
1    8151
0     425
Name: predi

In [22]:
best_result = best_df_list[0]

for index, row in best_df_list[0].iterrows():
    t_row_sum = 0
    for i in best_df_list:
        t_row_sum += i.loc[index].prediction
    if t_row_sum> 50:
        best_result.loc[index].prediction = 1
    else:
        best_result.loc[index].prediction = 0

print(best_result.prediction.value_counts())




1    8148
0     428
Name: prediction, dtype: int64


In [24]:
best_result_from_10 = best_df_list[0]
best_model_states = [3,6,12,13,33,37,45,49,60,96,78]
for index, row in best_df_list[0].iterrows():
    t_row_sum = 0
    for i in best_model_states:
        t_row_sum += best_df_list[i].loc[index].prediction
    if t_row_sum> 5:
        best_result_from_10.loc[index].prediction = 1
    else:
        best_result_from_10.loc[index].prediction = 0
print(best_result_from_10.prediction.value_counts())

1    8149
0     427
Name: prediction, dtype: int64


In [27]:
best_df_list_bagging = []
test_list = []
statesList = [3,6,12,13,33,37,45,49,60,96,78]
for i in statesList:    
    kf = KFold(n_splits=10, random_state=i, shuffle=True)


    kf.get_n_splits(features_df)
    test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                                   index_col="tripid")

    test_features_df = formatdata(test_features_df)
    powerAll(test_features_df, exceptList,power)

    bestscore = 0
    bestModelXX = ""
    test_probas_state_best =''


    for train_index, test_index in kf.split(features_df):

        X_train, X_eval, y_train, y_eval = features_df.iloc[train_index], features_df.iloc[test_index],labels_df.iloc[train_index], labels_df.iloc[test_index]
        full_pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("estimators", estimators),
        ])
        full_pipeline.fit(X_train, y_train)
        preds = full_pipeline.predict(X_eval)
        y_preds = pd.DataFrame(
            preds,
            index = y_eval.index
        )
        test_probas = full_pipeline.predict(test_features_df)
        test_preds = pd.DataFrame(
            {'prediction':test_probas},
            index = test_features_df.index
        )
        f1 = f1_score(y_eval, y_preds)
        if(f1 > bestscore):
            bestscore = f1
            bestModelXX = full_pipeline
            test_probas_state_best = full_pipeline.predict(test_features_df)

    test_probas_state_best_test = bestModelXX.predict(test_features_df)
    test_preds_state_best = pd.DataFrame(
        {'prediction':test_probas_state_best},
        index = test_features_df.index
    )
    test_preds_state_best_test = pd.DataFrame(
        {'prediction':test_probas_state_best_test},
        index = test_features_df.index
    )
    best_df_list_bagging.append(test_preds_state_best)
    test_list.append(test_preds_state_best_test)
    
    print("original",test_preds_state_best.prediction.value_counts())
    print("test",test_preds_state_best_test.prediction.value_counts())
    print('bestscore', bestscore)
    print("state", i)
    print('')

print("done")

original 1    8161
0     415
Name: prediction, dtype: int64
test 1    8161
0     415
Name: prediction, dtype: int64
bestscore 0.9825119236883942
state 3

original 1    8168
0     408
Name: prediction, dtype: int64
test 1    8163
0     413
Name: prediction, dtype: int64
bestscore 0.9803416048984853
state 6

original 1    8167
0     409
Name: prediction, dtype: int64
test 1    8157
0     419
Name: prediction, dtype: int64
bestscore 0.9810964083175803
state 12

original 1    8162
0     414
Name: prediction, dtype: int64
test 1    8150
0     426
Name: prediction, dtype: int64
bestscore 0.9818065751675711
state 13

original 1    8161
0     415
Name: prediction, dtype: int64
test 1    8161
0     415
Name: prediction, dtype: int64
bestscore 0.9829974811083123
state 33

original 1    8162
0     414
Name: prediction, dtype: int64
test 1    8162
0     414
Name: prediction, dtype: int64
bestscore 0.9835338822039265
state 37

original 1    8158
0     418
Name: prediction, dtype: int64
test 1    81

In [28]:
best_result_from_10_bagging = best_df_list_bagging[0]
for index, row in best_df_list_bagging[0].iterrows():
    t_row_sum = 0
    for i in best_df_list_bagging:
        t_row_sum += i.loc[index].prediction
    if t_row_sum> 5:
        best_result_from_10_bagging.loc[index].prediction = 1
    else:
        best_result_from_10_bagging.loc[index].prediction = 0
print(best_result_from_10_bagging.prediction.value_counts())

1    8165
0     411
Name: prediction, dtype: int64


In [30]:
best_result_from_10_bagging_test = test_list[0]
for index, row in test_list[0].iterrows():
    t_row_sum = 0
    for i in test_list:
        t_row_sum += i.loc[index].prediction
    if t_row_sum> 5:
        best_result_from_10_bagging_test.loc[index].prediction = 1
    else:
        best_result_from_10_bagging_test.loc[index].prediction = 0
print(best_result_from_10_bagging_test.prediction.value_counts())

1    8162
0     414
Name: prediction, dtype: int64


In [31]:
#0.9828134945894335 state 3 k=10
#0.9816188326346339 state 6 k=10
best_result_from_10_bagging_test.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [ ]:

solver = 'lbfgs'
alpha = 0.2
hidden_layer_sizes=(10,2 )
random_state=2
max_iter= 10000000
activation='tanh'
estimatorsX=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
                         max_iter=max_iter, activation=activation)

min_child_weight=1.5
max_depth=5
gamma=2.5
colsample_bytree=0.8
subsample=0.8
n_estimators=200
threads = 4
estimators = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )
params = { 'max_features': [0.8,0.85,0.9,0.95,1.0],'max_samples':[0.6,0.65,0.7,0.75,0.8]}

#estimatorsY = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=6,learning_rate=0.2,
 #                          n_estimators=1000,random_state=42)

#estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=0.65,
 #                            max_features=1.0, random_state=1,bootstrap=False)
#estimators = VotingClassifier( [('xgb', estimatorsX), ('mlp', estimatorsY)], weights=[1,1])
#estimators = GridSearchCV(estimator=estimatorsB, param_grid=params, cv=5)


In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=2
)
smt = SMOTE(sampling_strategy='all', random_state=None, k_neighbors=5, n_jobs=1)
svmus = SVMSMOTE(sampling_strategy='auto',  k_neighbors=10, random_state=42,
                                n_jobs=4, out_step=0.1)
smottomek = SMOTETomek(sampling_strategy ='all', random_state=2, n_jobs=1)

#X_train, y_train = svmus.fit_resample(X_train, y_train)
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

full_pipeline.fit(X_train, y_train)
preds = full_pipeline.predict(X_eval)
y_preds = pd.DataFrame(
    preds,
    index = y_eval.index
)

f1 = f1_score(y_eval, y_preds)
print(f1)

In [ ]:
#print(full_pipeline.named_steps['estimators'].best_params_, full_pipeline.named_steps['estimators'].best_score_)
print(f1)

In [ ]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
powerAll(test_features_df, exceptList,power)


test_probas = full_pipeline.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

test_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

print(confusion_matrix(y_eval, y_preds))
print(test_preds.prediction.value_counts())
print(f1)

In [ ]:
print(confusion_matrix(y_eval, y_preds))
print(test_preds.prediction.value_counts())
print(f1)

In [ ]:
#median
#0.9762502413593357
#1    8113
#0     463
#[[ 367  188]
 #[  58 5056]]

In [ ]:
#mean
#0.9754447022428461
#1    8124
#0     452
#[[ 370  185]
# [  69 5045]]

In [ ]:
#most_frequent
#0.9740121727369336
#1    8126
#0     450
#[[ 359  196]
# [  73 5041]]

In [ ]:
#constant
0.9746713070378964
#1    8123
#0     453
#[[ 366  189]
# [  73 5041]]